In [ ]:
pip install transformers

In [ ]:
pip install openpyxl

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_darija_to_eng = AutoTokenizer.from_pretrained("AnasAber/seamless-darija-eng")
model_darija_to_eng = AutoModelForSeq2SeqLM.from_pretrained("AnasAber/seamless-darija-eng")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.17M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.4M [00:00<?, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/483M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Instantiating a decoder SeamlessM4Tv2Attention without passing `layer_idx` is not recommended and will lead to errors during the forward call, if caching is used. Please make sure to provide a `layer_idx` when creating this class.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json: 0.00B [00:00, ?B/s]

In [ ]:
def translate_from_ary_to_en(text, src_lang, tgt_lang):
    inputs = tokenizer_darija_to_eng(text, return_tensors="pt")
    translated = model_darija_to_eng.generate(**inputs, forced_bos_token_id=tokenizer_darija_to_eng.convert_tokens_to_ids(tgt_lang),max_length=256,num_beams=5, early_stopping=True)
    return tokenizer_darija_to_eng.batch_decode(translated, skip_special_tokens=True)[0]

# Darija to English

darija_text = " اليوم الجو زوين"
english_translation = translate_from_ary_to_en(darija_text, src_lang="ary", tgt_lang="eng_Latn")
print("from darija to english")
print(english_translation)

You must either specify a `tgt_lang` or pass a correct `text_decoder_input_ids` to get
                a correct generation, otherwise the generation will probably make no sense.
Both `max_new_tokens` (=256) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


from darija to english
today is a good day


In [ ]:
import pandas as pd

ds = pd.read_excel(r'/content/sample_data/pos_comments.xlsx')
comments = ds['comment'].tolist()

In [ ]:
filtred_comments = [c for c in comments if len(c.split()) <= 10]

In [ ]:
translated_comments = []
for comment in filtred_comments:
    translated_comment= translate_from_ary_to_en(comment, src_lang="ary", tgt_lang="eng_Latn")

    if translated_comment:
        translated_comments.append(translated_comment)
    else:
        print(f"Failed to process: {comment}")

In [ ]:
translated_comments

In [ ]:
from transformers import pipeline

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_eng_to_darija = AutoTokenizer.from_pretrained("alpha2002/eng_alpha_darija")
model_eng_to_darija = AutoModelForSeq2SeqLM.from_pretrained("alpha2002/eng_alpha_darija")

def translate_from_eng_to_darija(text, src_lang, tgt_lang):
    inputs_eng = tokenizer_eng_to_darija(text, return_tensors="pt")
    translated_dr =model_eng_to_darija.generate(**inputs_eng, forced_bos_token_id=tokenizer_eng_to_darija.convert_tokens_to_ids("ar_AR"),max_length=256,num_beams=5,early_stopping=True)
    return tokenizer_eng_to_darija.batch_decode(translated_dr, skip_special_tokens=True)[0]



# english to darija
eng_text = "today is a good day"
darija_translation = translate_from_eng_to_darija(eng_text, src_lang="en_XX", tgt_lang="ar_AR")
print("from english to darija")
print(darija_translation)


tokenizer_config.json:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.68k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/256 [00:00<?, ?B/s]

from english to darija
اليوم نهار زوين


In [ ]:
darija_translation = translate_from_eng_to_darija(translated_comments[1], src_lang="en_XX", tgt_lang="ar_AR")
print("from english to darija")
print(translated_comments[1])
print(darija_translation)

In [ ]:
augmented_dataset = []
for comment in translated_comments:
    augmented_comment= translate_from_eng_to_darija(comment, src_lang="en_XX", tgt_lang="ar_AR")

    if augmented_comment:
        augmented_dataset.append(augmented_comment)
    else:
        print(f"Failed to process: {comment}")

In [ ]:
augmented_dataset

In [ ]:
data = {"Original Comment": filtred_comments, "Augmented Comment": augmented_dataset}
df = pd.DataFrame(data)

In [ ]:
df.to_excel('aug_pos_cmts_byBackTranslation.xlsx')